In [ ]:
import os
import json
import cv2
import torch
import numpy as np
import PIL
from PIL import Image
from einops import rearrange
from video_vae import CausalVideoVAELossWrapper
from torchvision import transforms as pth_transforms
from torchvision.transforms.functional import InterpolationMode
from IPython.display import Image as ipython_image
from diffusers.utils import load_image, export_to_video, export_to_gif
from IPython.display import HTML

In [ ]:
model_path = "pyramid-flow-miniflux/causal_video_vae"   # The video-vae checkpoint dir
model_dtype = 'bf16'

device_id = 3
torch.cuda.set_device(device_id)

model = CausalVideoVAELossWrapper(
    model_path,
    model_dtype,
    interpolate=False, 
    add_discriminator=False,
)
model = model.to("cuda")

if model_dtype == "bf16":
    torch_dtype = torch.bfloat16 
elif model_dtype == "fp16":
    torch_dtype = torch.float16
else:
    torch_dtype = torch.float32

def image_transform(images, resize_width, resize_height):
    transform_list = pth_transforms.Compose([
        pth_transforms.Resize((resize_height, resize_width), InterpolationMode.BICUBIC, antialias=True),
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    return torch.stack([transform_list(image) for image in images])


def get_transform(width, height, new_width=None, new_height=None, resize=False,):
    transform_list = []

    if resize:
        if new_width is None:
            new_width = width // 8 * 8
        if new_height is None:
            new_height = height // 8 * 8
        transform_list.append(pth_transforms.Resize((new_height, new_width), InterpolationMode.BICUBIC, antialias=True))
    
    transform_list.extend([
        pth_transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    transform_list = pth_transforms.Compose(transform_list)

    return transform_list


def load_video_and_transform(video_path, frame_number, new_width=None, new_height=None, max_frames=600, sample_fps=24, resize=False):
    try:
        video_capture = cv2.VideoCapture(video_path)
        fps = video_capture.get(cv2.CAP_PROP_FPS)
        frames = []
        pil_frames = []
        while True:
            flag, frame = video_capture.read()
            if not flag:
                break
    
            pil_frames.append(np.ascontiguousarray(frame[:, :, ::-1]))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = torch.from_numpy(frame)
            frame = frame.permute(2, 0, 1)
            frames.append(frame)
            if len(frames) >= max_frames:
                break

        video_capture.release()
        interval = max(int(fps / sample_fps), 1)
        pil_frames = pil_frames[::interval][:frame_number]
        frames = frames[::interval][:frame_number]
        frames = torch.stack(frames).float() / 255
        width = frames.shape[-1]
        height = frames.shape[-2]
        video_transform = get_transform(width, height, new_width, new_height, resize=resize)
        frames = video_transform(frames)
        pil_frames = [Image.fromarray(frame).convert("RGB") for frame in pil_frames]

        if resize:
            if new_width is None:
                new_width = width // 32 * 32
            if new_height is None:
                new_height = height // 32 * 32
            pil_frames = [frame.resize((new_width or width, new_height or height), PIL.Image.BICUBIC) for frame in pil_frames]
        return frames, pil_frames
    except Exception:
        return None


def show_video(ori_path, rec_path, width="100%"):
    html = ''
    if ori_path is not None:
        html += f"""<video controls="" name="media" data-fullscreen-container="true" width="{width}">
        <source src="{ori_path}" type="video/mp4">
        </video>
        """
    
    html += f"""<video controls="" name="media" data-fullscreen-container="true" width="{width}">
    <source src="{rec_path}" type="video/mp4">
    </video>
    """
    return HTML(html)

### Image Reconstruction

In [ ]:
image_path = 'image_path'

image = Image.open(image_path).convert("RGB")
resize_width = image.width // 8 * 8
resize_height = image.height // 8 * 8
input_image_tensor = image_transform([image], resize_width, resize_height)
input_image_tensor = input_image_tensor.permute(1, 0, 2, 3).unsqueeze(0)

with torch.no_grad(), torch.cuda.amp.autocast(enabled=True, dtype=torch.bfloat16):
    latent = model.encode_latent(input_image_tensor.to("cuda"), sample=True)
    rec_images = model.decode_latent(latent)

display(image)
display(rec_images[0])

### Video Reconstruction

In [ ]:
video_path = 'video_path'

frame_number = 57   # x*8 + 1
width = 640
height = 384

video_frames_tensor, pil_video_frames = load_video_and_transform(video_path, frame_number, new_width=width, new_height=height, resize=True)
video_frames_tensor = video_frames_tensor.permute(1, 0, 2, 3).unsqueeze(0)
print(video_frames_tensor.shape)

with torch.no_grad(), torch.cuda.amp.autocast(enabled=True, dtype=torch.bfloat16):
    latent = model.encode_latent(video_frames_tensor.to("cuda"), sample=False, window_size=8, temporal_chunk=True)
    rec_frames = model.decode_latent(latent.float(), window_size=2, temporal_chunk=True)

export_to_video(pil_video_frames, './ori_video.mp4', fps=24)
export_to_video(rec_frames, "./rec_video.mp4", fps=24)
show_video('./ori_video.mp4', "./rec_video.mp4", "60%")